# New JIT and AOT Compilers

[John Stachurski](http://johnstachurski.net)

In [ ]:
!pip install quantecon

In [ ]:
import numpy as np
from numba import jit
from quantecon.util import tic, toc
import matplotlib.pyplot as plt

## Problem 1: A Time Series Model

Consider the time series model

$$ x_{t+1} = \alpha x_t (1 - x_t) $$

Let's set $\alpha = 4$

In [ ]:
α = 4

Here's a typical time series:

In [ ]:
n = 200
x =  np.empty(n)
x[0] = 0.2
for t in range(n-1):
    x[t+1] = α * x[t] * (1 - x[t])
    
plt.plot(x)
plt.show()

## Python Test

Here's a function that simulates for `n` periods, starting from `x0`, and returns **only the final** value:

In [ ]:
def quad(x0, n):
    x = x0
    for i in range(1, n):
        x = α * x * (1 - x)
    return x

Let's see how fast this runs:

In [ ]:
n = 10_000_000

In [ ]:
tic()
x = quad(0.2, n)
toc()

## Fortran Test

Now let's try this in Fortran.

Note --- this step is intended to be a demo and will only execute if

* you have the file `fastquad.f90` in your pwd
* you have a Fortran compiler installed and modify the compilation code below appropriately

In [ ]:
%%file fortran_quad.f90

PURE FUNCTION QUAD(X0, N)
 IMPLICIT NONE
 INTEGER, PARAMETER :: DP=KIND(0.d0)                           
 REAL(dp), INTENT(IN) :: X0
 REAL(dp) :: QUAD
 INTEGER :: I
 INTEGER, INTENT(IN) :: N
 QUAD = X0
 DO I = 1, N - 1                                                
  QUAD = 4.0_dp * QUAD * real(1.0_dp - QUAD, dp)
 END DO
 RETURN
END FUNCTION QUAD

PROGRAM MAIN
 IMPLICIT NONE
 INTEGER, PARAMETER :: DP=KIND(0.d0)                          
 REAL(dp) :: START, FINISH, X, QUAD
 INTEGER :: N
 N = 10000000
 X = QUAD(0.2_dp, 10)
 CALL CPU_TIME(START)
 X = QUAD(0.2_dp, N)
 CALL CPU_TIME(FINISH)
 PRINT *,'last val = ', X
 PRINT *,'elapsed time = ', FINISH-START
END PROGRAM MAIN

In [ ]:
!gfortran -O3 fortran_quad.f90

In [ ]:
!./a.out

In [ ]:
!rm a.out

## Codon

Let's try `codon`, an AOT Python compiler

First we have to install it.

In [ ]:
!/bin/bash -c "$(curl -fsSL https://exaloop.io/install.sh)"

Now we write Python code to a file.

In [ ]:
%%file codon_quad.py

from time import time

n = 10_000_000
alpha = 4.0

def quad(x0, n):
    x = x0
    for i in range(1, n):
        x = alpha * x * (1 - x)
    return x


t0 = time()
x = quad(0.1, n)
t1 = time()
print(x)
print(t1 - t0)

Next we compile the Python code to build an executable.

In [ ]:
!codon build --release --exe codon_quad.py

Now let's run it.

In [ ]:
!./codon_quad

Tidying up:

In [ ]:
!rm codon_quad

## Python + Numba

Now let's replicate the calculations using Numba's JIT compiler.

Here's the Python function we want to speed up

In [ ]:
@jit
def quad(x0, n):
    x = x0
    for i in range(1, n):
        x = α * x * (1 - x)
    return x

This is the same as before except that we've targeted the function for JIT
compilation with `@jit`.

Let's see how fast it runs.

In [ ]:
tic()
x = quad(0.2, n)
toc()

In [ ]:
tic()
x = quad(0.2, n)
toc()